In [1]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:
import cv2
import os

def extract_frames(video_path, output_folder, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(total_frames // num_frames, 1)
    count = 0
    saved = 0

    while cap.isOpened() and saved < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if count % step == 0:
            filename = os.path.join(output_folder, f"{os.path.basename(video_path)[:-4]}_frame_{saved}.jpg")
            cv2.imwrite(filename, frame)
            saved += 1
        count += 1
    cap.release()

def extract_all_videos(input_folder, output_folder, num_frames=10):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):   # đúng nè
        if filename.endswith(".mp4") or filename.endswith(".avi") or filename.endswith(".mov"):
            video_path = os.path.join(input_folder, filename)
            extract_frames(video_path, output_folder, num_frames=num_frames)
            print(f"Đã cắt ảnh từ {filename}")

# Ví dụ dùng:

# Cắt từ Video Thật
extract_all_videos(
    "/content/drive/MyDrive/VideoReal",   # input: thư mục chứa video
    "/content/drive/MyDrive/LuuAnh/OutputReal",  # output: nơi lưu ảnh
    num_frames=250
)

# Cắt từ Video Giả
extract_all_videos(
    "/content/drive/MyDrive/VideoFake",
    "/content/drive/MyDrive/LuuAnh/OutputFake",
    num_frames=250
)


Đã cắt ảnh từ Unit1.mp4
Đã cắt ảnh từ WIN_20250309_20_33_48_Pro.mp4
Đã cắt ảnh từ WIN_20250309_21_09_41_Pro.mp4
Đã cắt ảnh từ WIN_20250317_21_25_34_Pro.mp4
Đã cắt ảnh từ WIN_20250414_21_15_35_Pro.mp4
Đã cắt ảnh từ WIN_20250414_21_39_48_Pro.mp4
Đã cắt ảnh từ WIN_20250423_21_28_38_Pro.mp4
Đã cắt ảnh từ cmagae5e878c739p7d4g.mp4
Đã cắt ảnh từ cn4652te878c73fkll3g.mp4
Đã cắt ảnh từ co15fmte878c7391e7i0.mp4
Đã cắt ảnh từ AI.mp4
Đã cắt ảnh từ bandicam 2025-02-19 15-41-17-079.mp4


In [ ]:
# Tải mô hình Xception đã pretrain (không bao gồm lớp đầu ra)
base_model = Xception(weights="imagenet", include_top=False, input_shape=(299, 299, 3))

# Thêm các lớp FC để phù hợp với bài toán Deepfake
x = Flatten()(base_model.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation="sigmoid")(x)  # Lớp đầu ra (0 = video thật, 1 = video giả)

# Xây dựng mô hình hoàn chỉnh
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Chuẩn bị dữ liệu từ thư mục chứa hình ảnh cắt từ video
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/LuuAnh",
    target_size=(299, 299),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/LuuAnh",
    target_size=(299, 299),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

# Huấn luyện mô hình
model.fit(train_generator, validation_data=val_generator, epochs=10)

# Lưu mô hình sau khi huấn luyện
model.save("deepfake_detector_xception.h5")


Found 3538 images belonging to 2 classes.
Found 883 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 635s 5s/step - accuracy: 0.9650 - loss: 1.3043 - val_accuracy: 1.0000 - val_loss: 6.2837e-06
Epoch 2/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 101s 910ms/step - accuracy: 1.0000 - loss: 1.0318e-06 - val_accuracy: 1.0000 - val_loss: 0.0030
Epoch 3/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 101s 910ms/step - accuracy: 1.0000 - loss: 1.7527e-07 - val_accuracy: 1.0000 - val_loss: 0.0067
Epoch 4/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 101s 903ms/step - accuracy: 1.0000 - loss: 2.0373e-07 - val_accuracy: 1.0000 - val_loss: 0.0083
Epoch 5/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 143s 911ms/step - accuracy: 1.0000 - loss: 3.3149e-08 - val_accuracy: 1.0000 - val_loss: 0.0081
Epoch 6/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 141s 907ms/step - accuracy: 1.0000 - loss: 7.4755e-08 - val_accuracy: 1.0000 - val_loss: 0.0077
Epoch 7/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 100s 900ms/step - accuracy: 1.0000 - loss: 2.0813e-07 - val_accuracy: 1.0000 - val_loss: 0.0071
Epoch 8/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 101s 906ms/

In [ ]:
from tensorflow.keras.models import load_model

# Tải mô hình đã huấn luyện
model_path = "/content/deepfake_detector_xception.h5"
model = load_model(model_path)

# In ra kiến trúc mô hình
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 149, 149,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 149, 149,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 147, 147,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 147, 147,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 147, 147,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 147, 147,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 147, 147,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 147, 147,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 147, 147,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 147, 147,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 74, 74,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 74, 74,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 74, 74,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 74, 74,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 74, 74,    │          0 │ add[0][0]       

 Total params: 125,720,107 (479.58 MB)

 Trainable params: 125,665,577 (479.38 MB)

 Non-trainable params: 54,528 (213.00 KB)

 Optimizer params: 2 (12.00 B)

In [ ]:
def extract_frames(video_path, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            frame = cv2.resize(frame, (299, 299))  # Xception yêu cầu kích thước 299x299
            frames.append(frame)
        frame_count += 1

    cap.release()
    return np.array(frames)


In [ ]:
def detect_deepfake(video_path):
    frames = extract_frames(video_path)
    frames = frames.astype("float32") / 255.0  # Chuẩn hóa dữ liệu

    predictions = model.predict(frames)
    avg_score = np.mean(predictions)  # Trung bình điểm dự đoán của tất cả khung hình

    if avg_score < 0.5:
        print(f"🔴 Video có khả năng là Deepfake! (Score: {avg_score:.2f})")
    else:
        print(f"🟢 Video có vẻ là thật! (Score: {avg_score:.2f})")

    # Hiển thị một số khung hình cùng với dự đoán
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    for i in range(5):
        axes[i].imshow(frames[i])
        axes[i].axis("off")
        axes[i].set_title(f"Score: {predictions[i][0]:.2f}")

    plt.show()



In [ ]:
video_path = "/content/drive/MyDrive/Colab Notebooks/Videonguoi1.mp4"
detect_deepfake(video_path)



NameError: name 'detect_deepfake' is not defined